In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
import collections
import nest_asyncio
#%reload_ext tensorboard
nest_asyncio.apply()
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense
#from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.datasets import mnist

In [2]:
# Neural network model in pytorch defined here

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
#        self.conv1 = nn.Conv2d(1, 20, 5, 1)
#        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(784, 10)
#        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = torch.from_numpy(np.array(x))
#        x = F.relu(self.conv1(x))
#        x = F.max_pool2d(x, 2, 2)
#        x = F.relu(self.conv2(x))
#        x = F.max_pool2d(x, 2, 2)
#        x = x.view(-1, 4*4*50)
#        x = F.relu(self.fc1(x))
#        x = self.fc2(x)
        return F.log_softmax(x, dim =1)

In [3]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data(only_digits=True)

In [4]:
class server:
    
    def __init__(self, numOfClients):
        self.numOfClients = numOfClients #Num of Client
        self.clientIds = list(range(0,self.numOfClients)) #Client List
        self.updateRoundNum = 0  # Update Round Number
        self.serverModel = self.createBaseModel() #Create a Keras Model for MNIST
        self.clientActive = [] #If client is available or not based on time - day/night
        self.clientCIF = []
        self.updateFromClients=[]
        self.__serverTestData_X=[]
        self.__serverTestData_Y=[]
        self.__serverTestData= self.setServerTestDataTFF(emnist_test)
        self.predictionAcc=[]
        self.predictionLoss=[]
        
        
    def createBaseModel(self):
        return Net()
    
    def setServerTestData(self,Xtest,Ytest):  # NOT USED IN THIS CASE
        self.__serverTestData_X=Xtest
        self.__serverTestData_Y=Ytest
        
    def setServerTestDataTFF(self,serverTestData):  #Creates a test set for server
        BATCH_SIZE = 100
        SHUFFLE_BUFFER = 100
        PREFETCH_BUFFER= 10

        def preprocess(dataset):
            def batch_format_fn(element):
                """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
                return collections.OrderedDict(
                    x=tf.reshape(element['pixels'], [-1, 784]),
                    y=tf.reshape(element['label'], [-1, 1]))

            return dataset.repeat(10).shuffle(SHUFFLE_BUFFER).batch(
              BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

        serverData = tfds.as_numpy(preprocess(serverTestData.create_tf_dataset_for_client(serverTestData.client_ids[30])))

        return serverData

    
    def initialBroadcast(self):
        for i in self.clientIds:
            clientName = "client_"+str(self.clientIds[i])  #Create Client Name
            #print(clientName)
            eval(clientName).setInitialModel(self.serverModel) #setInitialModel-> Method of client Class
            
    def getClientActiveStatus(self):  #if the client is available for update or not randomly set in client
        self.clientActive = []
        for i in self.clientIds:
            clientName = "client_"+str(self.clientIds[i])  #Create Client Name
            #print(clientName)
            if(eval(clientName).sendActiveStatus() == 1): #getActiveStatus() -> method of client class
                self.clientActive.append(i)
        
            
    def getClientCIF(self): # Get Client Importance Factor for active clients
        self.clientCIF=[]
        for i in self.clientActive:
            clientName = "client_"+str(i)
            c_cif = eval(clientName).sendCIF()   #sendCIF() - method of client
            self.clientCIF.append(c_cif)
        #print("Clients with Acceptable CIF: ", self.clientCIF)
    
    def getModelUpdateFromClients(self):
        print("--------------------------------------------\n","Round NO:",self.updateRoundNum)
        print("Active Clients: ",self.clientActive)
        print("CIF of Active Clients: ", self.clientCIF)
        print("Training Clients with CIF > 10")
        self.updateRoundNum +=1
        self.updateFromClients=[]
        #self.dataPointsClients=[]
        for i in range (0,len(self.clientActive)):
            if self.clientCIF[i] >= 10:  #Select Client only if CIF > some value 10 chosen for testing ONLY
                clientName = "client_"+str(int(self.clientActive[i]))
                self.updateFromClients.append(eval(clientName).sendClientUpdate()) #Get a list of updates from selected clients
                #print(updateFromClients)
        
        avgModelWeights = self.computeFedAvg(self.updateFromClients)
        #print("average model weights = \n",avgModelWeights)
        self.serverModel.set_weights(avgModelWeights)   
        
    def computeFedAvg(self,updates):  #Compute Federated Averaging from Available Clients
        totalDataPoints = 0
        scaleFactor=[]
        for i in range (0,len(updates)):
            totalDataPoints += updates[i][1]   #Sum the total Data Points on All Available Clients
            scaleFactor.append(updates[i][1])  #Store individual number datapoints for client
        
        
        scaleFactor = np.array(scaleFactor)/totalDataPoints #Create the scale factor
        
        sumOfAvgWeights = []*len(updates[0][0]) 
        
        for j in range (0,len(updates[0][0])): #range of layers
            k=np.zeros_like(updates[0][0][j])
            for i in range(0,len(updates)): #range of clients
                k=k+updates[i][0][j]*scaleFactor[i]
                #print("ShapeofK:",k.shape)
            sumOfAvgWeights.append(k)
        
        
        return sumOfAvgWeights   
    
    def testServerModel(self):
        testData = next(iter(self.__serverTestData))
        output = self.serverModel(testData['x'])
        loss = F.nll_loss(output, testData['y']).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct = pred.eq(testData['y'].view_as(pred)).sum().item()
        accuracy = correct/len(testData)
        return loss, accuracy
      
    def updateAllClients(self):
        for i in self.clientIds:
            clientName = "client_"+str(self.clientIds[i])  #Create Client Name
            eval(clientName).setModelUpdateWeights(self.serverModel.get_weights()) 



In [5]:
class client:
    
    def __init__(self,ID):
        self.id = ID
        self.__clientModel = Net()
        self.__clientModelWeights=[]
        self.activeStatus = 0
        self.__clientDataX=[] #Private to Client
        self.__clientDataY=[]
        self.clientCIF=0
        self.SNR = 0
        self.epochs = 10
        self.lr = 0.01
        self.__clientData = self.__clientDataFromTFF(emnist_train)
    
    
    #Methods Private to client
    
    def __clientDataFromTFF(self,tffDataset):
        NUM_EPOCHS = self.epochs
        BATCH_SIZE = 100
        SHUFFLE_BUFFER = 100
        PREFETCH_BUFFER= 10

        def preprocess(dataset):
            def batch_format_fn(element):
                """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
                return collections.OrderedDict(
                    x=tf.reshape(element['pixels'], [-1, 784]),
                    y=tf.reshape(element['label'], [-1, 1]))

            return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
              BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)
        
        return tfds.as_numpy(preprocess(tffDataset.create_tf_dataset_for_client(tffDataset.client_ids[self.id])))
        
    
    def __setActiveStatus(self): 
        self.activeStatus = int(np.random.randint(0,2,1))
        #self.activeStatus = 1
    
    def __setCIF(self):
        self.clientCIF = int(np.random.randint(0,50,1))  # Random Number between 0 and 49 for vheck purpose only
        #self.clientCIF = 20
        
    def __getTrainData(self):
        return self.__clientDataX, self.__clientDataY  # Just for Testing will be expanded to select meaningful data
    
    #Public Methods
    def sendActiveStatus(self):
        self.__setActiveStatus()
        return self.activeStatus
    
    def sendCIF(self):
        self.__setCIF()
        return self.clientCIF
    
    def setInitialModel(self,model):
        self.__clientModel=model
        print(self.__clientModel)
         
    def setClientData(self,X,Y):
        self.__clientDataX = X
        self.__clientDataY = Y
   

    def sendClientUpdate(self):
        print("->",self.id,end=" ")
        optimizer = optim.SGD(self.__clientModel.parameters(), lr = self.lr)
        #print(self.__clientDataX.shape)
        #print(self.__clientDataY.shape)
        if len(self.__clientDataX) > 0:
            for epoch in range(self.epochs):
                optimizer.zero_grad()
                output = self.__clientModel(self.__clientDataX)
                loss = F.nll_loss(output, self.__clientDataY)
                loss.backward()
                optimizer.step()

            updatedWeights = []
            for param in self.__clientModel.parameters():
                updatedWeights.append(param)
        
        else:
            currentData = next(iter(self.__clientData))
            for epoch in range(self.epochs):
                optimizer.zero_grad()
                output = self.__clientModel(self.__clientDataX)
                loss = F.nll_loss(output, self.__clientDataY)
                loss.backward()
                optimizer.step()

            updatedWeights = []
            for param in self.__clientModel.parameters():
                updatedWeights.append(param)
       
        
        numOfDataPoints = len(currentData['x'])   #<-- Set the correct type to get the number of datapoints in the current update round

        return (updatedWeights,numOfDataPoints)
    
    def setModelUpdateWeights(self,modelUpdateWeights):
        self.__clientModel.set_weights(modelUpdateWeights)
        #self.__clientModel.
        
    def plotClientData(self):
        #print("Y_Data",self.__clientDataY)
        id0=[]
        id1=[]
        id2=[]
        id3=[]
        id4=[]
        id5=[]
        id6=[]
        id7=[]
        id8=[]
        id9=[]
        for i in range (0,len(self.__clientData['x'])):
            for j in range (0,10):
                if j==self.__clientData['y']:
                    eval('id'+str(j)).append(i)
        xAxis=['0','1','2','3','4','5','6','7','8','9']
        yAxis=[len(id0),len(id1),len(id2),len(id3),len(id4),len(id5),len(id6),len(id7),len(id8),len(id9)]
        plt.bar(xAxis,yAxis)
        title="Data for Client"+str(self.id)
        plt.title(title)
        plt.show()

## Initializing Clients

In [6]:
#Populate Clients
N = 100

client_0=client(0)
client_1=client(1)
client_2=client(2)
client_3=client(3)
client_4=client(4)
client_5=client(5)
client_6=client(6)
client_7=client(7)
client_8=client(8)
client_9=client(9)
client_10=client(10)
client_11=client(11)
client_12=client(12)
client_13=client(13)
client_14=client(14)
client_15=client(15)
client_16=client(16)
client_17=client(17)
client_18=client(18)
client_19=client(19)
client_20=client(20)
client_21=client(21)
client_22=client(22)
client_23=client(23)
client_24=client(24)
client_25=client(25)
client_26=client(26)
client_27=client(27)
client_28=client(28)
client_29=client(29)
client_30=client(30)
client_31=client(31)
client_32=client(32)
client_33=client(33)
client_34=client(34)
client_35=client(35)
client_36=client(36)
client_37=client(37)
client_38=client(38)
client_39=client(39)
client_40=client(40)
client_41=client(41)
client_42=client(42)
client_43=client(43)
client_44=client(44)
client_45=client(45)
client_46=client(46)
client_47=client(47)
client_48=client(48)
client_49=client(49)
client_50=client(50)
client_51=client(51)
client_52=client(52)
client_53=client(53)
client_54=client(54)
client_55=client(55)
client_56=client(56)
client_57=client(57)
client_58=client(58)
client_59=client(59)
client_60=client(60)
client_61=client(61)
client_62=client(62)
client_63=client(63)
client_64=client(64)
client_65=client(65)
client_66=client(66)
client_67=client(67)
client_68=client(68)
client_69=client(69)
client_70=client(70)
client_71=client(71)
client_72=client(72)
client_73=client(73)
client_74=client(74)
client_75=client(75)
client_76=client(76)
client_77=client(77)
client_78=client(78)
client_79=client(79)
client_80=client(80)
client_81=client(81)
client_82=client(82)
client_83=client(83)
client_84=client(84)
client_85=client(85)
client_86=client(86)
client_87=client(87)
client_88=client(88)
client_89=client(89)
client_90=client(90)
client_91=client(91)
client_92=client(92)
client_93=client(93)
client_94=client(94)
client_95=client(95)
client_96=client(96)
client_97=client(97)
client_98=client(98)
client_99=client(99)




### Initializing Sever

In [7]:

serverFA = server(N)   # N = Number of Client


In [8]:
serverFA.initialBroadcast()

Net(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)
Net(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)
Net(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)
Net(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)
Net(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)
Net(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)
Net(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)
Net(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)
Net(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)
Net(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)
Net(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)
Net(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)
Net(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)
Net(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)
Net(
  (fc1): Linear(in_features=784, out_featur

In [9]:
numberOfTraingRounds = 10
trainingMetrics=[]*numberOfTraingRounds
for i in range(0,numberOfTraingRounds):
    serverFA.getClientActiveStatus()
    serverFA.getClientCIF()
    serverFA.getModelUpdateFromClients()
    trainingMetrics.append(serverFA.testServerModel())
    serverFA.updateAllClients()


--------------------------------------------
 Round NO: 0
Active Clients:  [0, 1, 2, 5, 6, 8, 9, 10, 12, 13, 14, 17, 19, 21, 22, 27, 28, 29, 31, 32, 34, 37, 40, 42, 44, 45, 49, 50, 56, 65, 66, 67, 70, 72, 76, 78, 80, 81, 82, 84, 86, 89, 91, 92, 93, 95, 96]
CIF of Active Clients:  [33, 29, 4, 19, 3, 39, 20, 20, 7, 2, 42, 19, 29, 21, 4, 20, 18, 0, 43, 10, 31, 4, 44, 24, 26, 36, 13, 28, 48, 3, 31, 44, 0, 35, 46, 45, 8, 35, 47, 18, 13, 43, 22, 16, 37, 42, 14]
Training Clients with CIF > 10
-> 0 

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
plt.plot(trainingMetrics)

### Below are Rough cells ... donot run

In [ ]:
a.baseModel.set_weights(avgWeight)

In [ ]:
a.baseModel.get_weights()

In [ ]:
evaldata=serverFA.testServerModel()

In [ ]:
type(evaldata)

In [ ]:
evaldata

In [ ]:
#Populate Clients
N = 100

client_0=client(0)
client_1=client(1)
client_2=client(2)
client_3=client(3)
client_4=client(4)
client_5=client(5)
client_6=client(6)
client_7=client(7)
client_8=client(8)
client_9=client(9)
client_10=client(10)
client_11=client(11)
client_12=client(12)
client_13=client(13)
client_14=client(14)
client_15=client(15)
client_16=client(16)
client_17=client(17)
client_18=client(18)
client_19=client(19)
client_20=client(20)
client_21=client(21)
client_22=client(22)
client_23=client(23)
client_24=client(24)
client_25=client(25)
client_26=client(26)
client_27=client(27)
client_28=client(28)
client_29=client(29)
client_30=client(30)
client_31=client(31)
client_32=client(32)
client_33=client(33)
client_34=client(34)
client_35=client(35)
client_36=client(36)
client_37=client(37)
client_38=client(38)
client_39=client(39)
client_40=client(40)
client_41=client(41)
client_42=client(42)
client_43=client(43)
client_44=client(44)
client_45=client(45)
client_46=client(46)
client_47=client(47)
client_48=client(48)
client_49=client(49)
client_50=client(50)
client_51=client(51)
client_52=client(52)
client_53=client(53)
client_54=client(54)
client_55=client(55)
client_56=client(56)
client_57=client(57)
client_58=client(58)
client_59=client(59)
client_60=client(60)
client_61=client(61)
client_62=client(62)
client_63=client(63)
client_64=client(64)
client_65=client(65)
client_66=client(66)
client_67=client(67)
client_68=client(68)
client_69=client(69)
client_70=client(70)
client_71=client(71)
client_72=client(72)
client_73=client(73)
client_74=client(74)
client_75=client(75)
client_76=client(76)
client_77=client(77)
client_78=client(78)
client_79=client(79)
client_80=client(80)
client_81=client(81)
client_82=client(82)
client_83=client(83)
client_84=client(84)
client_85=client(85)
client_86=client(86)
client_87=client(87)
client_88=client(88)
client_89=client(89)
client_90=client(90)
client_91=client(91)
client_92=client(92)
client_93=client(93)
client_94=client(94)
client_95=client(95)
client_96=client(96)
client_97=client(97)
client_98=client(98)
client_99=client(99)



def distributeDataToClient(numOfClients,X_train,Y_train):
    
        for i in range (0,numOfClients):
            np.random.seed(i*10)
            numOfClasses = int(np.random.randint(4,10,1))
            classList = np.random.randint(0,10,numOfClasses)
            print(classList)
            for k in range (0,numOfClasses):
                classID = int(classList[k])
                classIDLen = len(eval("id"+str(classID)))
                #print(classID,":",classIDLen)
                numOfSamplesInClass= int(np.random.randint(200,1000,1))
                #print(numOfSamplesInClass)
                Y_data = np.empty([numOfSamplesInClass,])
                X_data= np.empty([numOfSamplesInClass,784])
                Y_data = np.random.choice(Y_train,numOfSamplesInClass,replace=False)
                #print(Y_data)
                for n in range(Y_data.size):
                    X_data[n] = np.copy(X_train[n])

            clientName = "client_"+str(i)
            c_data = (X_data,Y_data)
            eval(clientName).setClientData(X_data,Y_data)
            print(X_data.shape,Y_data.shape)

def distributeDataToServer(serverName,X_test,Y_test):
    serverName.setServerTestData(X_test,Y_test)
    
distributeDataToClient(20,originalTrainX,originalTrainY)
distributeDataToServer(serverFA,originalTestX,originalTestY)
        

In [ ]:
("client_",1)

In [ ]:
client_0.

In [ ]:
client_0.sendClientUpdate()

In [ ]:
client_0=client(0)
client_1=client(1)
client_2=client(2)
client_3=client(3)
client_4=client(4)
client_5=client(5)
client_6=client(6)
client_7=client(7)
client_8=client(8)
client_9=client(9)
client_10=client(10)
client_11=client(11)
client_12=client(12)
client_13=client(13)
client_14=client(14)
client_15=client(15)
client_16=client(16)
client_17=client(17)
client_18=client(18)
client_19=client(19)
client_20=client(20)
client_21=client(21)
client_22=client(22)
client_23=client(23)
client_24=client(24)
client_25=client(25)
client_26=client(26)
client_27=client(27)
client_28=client(28)
client_29=client(29)
client_30=client(30)
client_31=client(31)
client_32=client(32)
client_33=client(33)
client_34=client(34)
client_35=client(35)
client_36=client(36)
client_37=client(37)
client_38=client(38)
client_39=client(39)
client_40=client(40)
client_41=client(41)
client_42=client(42)
client_43=client(43)
client_44=client(44)
client_45=client(45)
client_46=client(46)
client_47=client(47)
client_48=client(48)
client_49=client(49)
client_50=client(50)
client_51=client(51)
client_52=client(52)
client_53=client(53)
client_54=client(54)
client_55=client(55)
client_56=client(56)
client_57=client(57)
client_58=client(58)
client_59=client(59)
client_60=client(60)
client_61=client(61)
client_62=client(62)
client_63=client(63)
client_64=client(64)
client_65=client(65)
client_66=client(66)
client_67=client(67)
client_68=client(68)
client_69=client(69)
client_70=client(70)
client_71=client(71)
client_72=client(72)
client_73=client(73)
client_74=client(74)
client_75=client(75)
client_76=client(76)
client_77=client(77)
client_78=client(78)
client_79=client(79)
client_80=client(80)
client_81=client(81)
client_82=client(82)
client_83=client(83)
client_84=client(84)
client_85=client(85)
client_86=client(86)
client_87=client(87)
client_88=client(88)
client_89=client(89)
client_90=client(90)
client_91=client(91)
client_92=client(92)
client_93=client(93)
client_94=client(94)
client_95=client(95)
client_96=client(96)
client_97=client(97)
client_98=client(98)
client_99=client(99)



client_10=client(10)
client_11=client(11)
client_12=client(12)
client_13=client(13)
client_14=client(14)
client_15=client(15)
client_16=client(16)
client_17=client(17)
client_18=client(18)
client_19=client(19)

### --> SNR - Get Value the Channel Importance - Transmit Power, Additional Gaussian Noice, Channel Rayliegh Fading (hk) 

### -->Data Augmentation at Every Round
: Client 0 - 6000 Data, For first round - 3000 - specifying the epochs, batch_size =32 (Default) model.fit()
3400 - Calculate the Entropy - CIF along with the SNR- Importance of the update Training - additional 400 data
4000

### --> Code Benchmarking use EMNIST dataset and check the code w.r.t TFF solution <- 20TH NOV

### --> Check on the leaf dataset - LSTM , CNN, 

### --> Second order optimizer - implement using adahessian < change the model pytorch

### --> comparisons with Just SNR, SNR with Data importance, with and without adahessian, 

### --> check for convergence vs number of users

### --> How will the code perform if the model changed to CNN